# Solutions to the Assessment Problems for Computer Infrastructure Module

In [27]:
#import pandas
import pandas as pd
#import yfinance package
import yfinance as yf
#import datetime
import datetime as dt

## Problem 1: Data from yfinance

In [37]:
# function get_data()
def get_data(tickers, start = dt.datetime.now() - dt.timedelta(days=5), end = dt.datetime.now(), interval = "1d"):
    data = yf.download(tickers, start=start, end=end, interval=interval, auto_adjust=True)
    print(data.head())
    data.to_csv("data.csv", sep = ",")


In [38]:
get_data(['META', 'AAPL', 'AMZN', 'NFLX' , 'GOOG'])

[*********************100%***********************]  5 of 5 completed

Price            Close                                                   \
Ticker            AAPL        AMZN        GOOG        META         NFLX   
Date                                                                      
2025-10-20  262.239990  216.479996  257.019989  732.169983  1238.560059   
2025-10-21  262.769989  222.029999  251.339996  733.270020  1241.349976   
2025-10-22  258.450012  217.949997  252.529999  733.409973  1116.369995   
2025-10-23  259.579987  221.089996  253.729996  734.000000  1113.589966   
2025-10-24  263.459991  224.389999  261.109985  739.272400  1098.680054   

Price             High                                                   ...  \
Ticker            AAPL        AMZN        GOOG        META         NFLX  ...   
Date                                                                     ...   
2025-10-20  264.380005  216.690002  257.880005  733.770020  1247.760010  ...   
2025-10-21  265.290009  223.320007  255.380005  738.500000  1248.599976  ...   

In [31]:
print(dt.datetime.now())

2025-10-24 20:16:34.230670


https://mimo.org/glossary/python/datetime

## Problem 2: Plotting Data

## Problem 3: Script

## Problem 4: Automation

## End